In [1]:
%matplotlib inline

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.preprocessing as sp

In [3]:
Dataset = np.load('Dataset.npy')

In [4]:
model = tf.keras.models.load_model('Final_model.h5')

In [5]:
Data_False = np.load('Data_False.npy')
X_False = Data_False[:9]
Y_False = Data_False[-3:]

X_train     = np.load('X_train.npy')
X_train_scl = np.load('X_train_scl.npy')

Y_train_stdev = np.load('Y_train_stdev.npy')
Y_train_mean  = np.load('Y_train_mean.npy')

In [6]:
def Standardization(Data):
    scaler = sp.StandardScaler().fit(Data)
    scl = scaler.transform(Data)
    return scl , scaler

def Predict(X_train,X_Try):
    X_train = np.vstack([X_train,X_Try])
    X_train_scl = Standardization(X_train)[0]
    Y_predict_scl = model.predict(X_train_scl)
    Y_predict = Y_predict_scl * Y_train_stdev + Y_train_mean
    X_rec = X_train[-1,:]
    Y_rec = Y_predict[-1,1]
    return X_rec , Y_rec

def Err(X_Try,X_False):
    Err1 = ((X_Try[1] - X_False[1]) ** 2) * (1/0.131695)
    Err2 = ((X_Try[5] - X_False[5]) ** 2) * (1/0.340098)
    Err3 = ((X_Try[6] - X_False[6]) ** 2) * (1/0.159337)
    Err4 = ((X_Try[7] - X_False[7]) ** 2) * (1/0.225232)
    Error = Err1 + Err2 + Err3 + Err4
    return Error

In [7]:
X_Try = X_False.copy()
Error_0 = 100000000000000
print(X_False)

for i in range(100):
    X_Try[1] = X_False[1] + 1.0  * random.uniform(-1,1) #FR
    X_Try[5] = X_False[5] + 1  * random.uniform(-1,1) #RR
    X_Try[6] = X_False[6] + 100.0  * random.uniform(-1,1) #B
    X_Try[7] = X_False[7] + 1000.0 * random.uniform(-1,1) #QR
    XME_D = Predict(X_train,X_Try)[1]
    print(XME_D)
    if XME_D >= 0.9:
        Error = Err(X_Try,X_False)
        print(Error)
        if Error < Error_0:
            X_ans = X_Try.copy()
            Error_0 = Error
            XME_D_ans = XME_D
print(X_ans)
print(XME_D_ans)

[ 2.01600000e+02  2.63000000e+00  3.80000000e+01  2.33000000e+01
 -4.16123817e+03  1.23800000e+00  9.73300000e+01  2.72111058e+03
 -2.52787842e+03]
0.9168018292800999
72043.91115701759
0.5428173800681917
0.6538275350147592
0.5873184818337527
0.7378230056663356
0.7275007497947955
0.770955595308778
0.7322587500184697
0.753896898043299
0.8688092543702628
0.8957448046027744
0.6387486529130806
0.8818923959265436
0.794491248319436
0.6018670820824286
0.9520029985491663
172747.81341942557
0.9358690586721005
631662.5793551956
0.9103190626872594
3999319.6080608303
0.9360680998063213
2389263.857134058
0.8195099979177688
0.6361001310257877
0.5545383414461815
0.6229199088610258
0.614796739599938
0.733948045201257
0.8222041801989385
0.654765634079534
0.8841512761469884
0.9261603543868028
4123014.7441870654
0.6456649580260299
0.7447082664905568
0.6947600768737152
0.8175827429849485
0.6145587798499048
0.5946802595465337
0.7131229030905994
0.9468308749314321
23753.54038069177
0.7894102401371603
0.67669